# TME 4 : Suffix Trees
Lev Savolskiy
21241759

## Recheche de pattern (motifs) en utilisant les suffix trees

Nous allons utiliser l'algorithme suffix-tree pour une recherche rapide et éfficace de motifs. Un suffix-tree est construit à partir d'un jeux de séquences, ensuite nous pouvons rechercher le motif en temps O(k) où k est la longueur du motif.

1\. Nous allons réutiliser les fonctions du TMEs precedents pour générer ``t`` séquences artificielles de taille ``n``, et implanter dans chaque séquence un motif de taille ``k`` à des positions aléatoires avec ``v`` substitutions choisies aléatoirement. Cependant, les ``t`` séquences artificielles initiales (sans implantation) ainsi que le motif initial (sans variation/mutation) doivent être générées une seule fois. Ensuite, selon chaque question, nous introduisons des différentes variation au motif initial et les implantons dans les séquences initiales afin de générer des nouveau jeux de données. 

In [1]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=8 #taille de motif
v=0 #nb de positions variables dans le motif
t=5 #nb de sequences
n=100 #longuer des sequence
f=0.9


1.1\. Generer les séquences artificielles initiales et implanter un motif (sans variation, v=0)

In [2]:

nuc = ('A', 'C', 'G', 'T')

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    sequence = "".join([random.choice(nuc) for _ in range(n)])
    if upper:
        return sequence
    return sequence.lower()

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)

    nbPos_real = min(nbpos, len(motif))
    allPos = list(range(len(motif)))

    for _ in range(nbPos_real):
        index = random.choice(range(len(allPos)))
        changeIndex = allPos[index]
        del allPos[index]

        nv_L = generateRandomSequence(1, upper)
        motifM[changeIndex] = nv_L
    return "".join(motifM)        


def implantMotifVar(k:int, v:int, t:int, n:int, f:float, adn = False):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    ##adn false or true, parce que c'est pas grave pour notre TME si on travaille avec des sequences pregenerés ou on les genere ici
    
    if (not adn):
        adn = []
        for i in range(t):
            adn.append(generateRandomSequence(n - k))
    
    sequences = adn

    motif = generateRandomSequence(k)
    print(motif.upper())
    motif_new = motif
    for i in range(t):
        base_seq = adn[i]
        if random.random() < f:
            motif_new = modifierMotif(motif, v)
        sequences[i] = (insertMotif(base_seq, motif_new, random.choice(range(len(base_seq)))))
        motif_new = motif
    
    return sequences


adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn = [s.upper() for s in adn]
print (adn)

ATAGTCGA
['tccatgtcgcggtctgagggaccttgtccggttctccctcgaaagctcaggactaccagattgtaaggatgaaatagtcgaagatttccatgggtaccct', 'ggaatattaccatcttagagatttacagttcctggtactcggcccacccgaatagtcgaggcgtagtcgaatttcgtctaacagctacagcaatctcaat', 'cagaacccgcgctcgatgaagaatgggctcgtacgtggccgcgttgcaaatagtcgaccccctcgcacgccatggcgagcaggcttctgttatgcgactg', 'tgacaacaaggacgttacatatagtcgagcaggctcgccaaactgaatttttccggggggttgtgaagtgctctttgagctcgagtcagatgaatacact', 'tagggccaaaccttgagaatcgtttcttggacctcagcgagcctttcgctccgcccaaccttgcagcaggagaaaatcgtatagtcgaggcgggaaatgt']
['TCCATGTCGCGGTCTGAGGGACCTTGTCCGGTTCTCCCTCGAAAGCTCAGGACTACCAGATTGTAAGGATGAAATAGTCGAAGATTTCCATGGGTACCCT', 'GGAATATTACCATCTTAGAGATTTACAGTTCCTGGTACTCGGCCCACCCGAATAGTCGAGGCGTAGTCGAATTTCGTCTAACAGCTACAGCAATCTCAAT', 'CAGAACCCGCGCTCGATGAAGAATGGGCTCGTACGTGGCCGCGTTGCAAATAGTCGACCCCCTCGCACGCCATGGCGAGCAGGCTTCTGTTATGCGACTG', 'TGACAACAAGGACGTTACATATAGTCGAGCAGGCTCGCCAAACTGAATTTTTCCGGGGGGTTGTGAAGTGCTCTTTGAGCTCGAGTCAGATGAATACACT', 'TAGGGCCAAACCTTGAGAATCGTTTCTTGGACCTCAGCGAGCCTTTCGCTCC

2\. Définissez une fonction ``construct_tree`` pour construire un suffix tree à partir des séquences artificielles (après implantation) en utilisant le python package suffix-trees trouvable ici: https://pypi.org/project/suffix-trees/. Tester si votre fonction est capable de trouver le motif sans variation implanté.

In [3]:
!pip install suffix-trees


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from suffix_trees import STree

st = STree.STree("abcdefghab")
print(st.find("abc")) # 0
print(st.find_all("ab")) # [0, 8]

def construct_tree(sequences):
    """
    construire un abre de suffixe
    entrée1 : matrice de dimension txn avec les sequences
    sortie1 : arbre de suffixe
    REMARK: Vous devez concatener toutes les sequences de la matrice avant d'appeller la fonction STree
    """
    ns = ''.join(sequences[0:len(sequences)-1])
    st = STree.STree(ns)
    return st

tree = construct_tree(adn)
fix_motif = 'TGAAAATG' #changer pour un teste rapide
tree.find_all(fix_motif)


0
{8, 0}


{}

3\. Avant de chercher les motifs, implémentez ou reutilisez les fonctions pour générer tous les motifs (k-mer) possibles de taille k, en éliminant les motifs peu complexe pour éviter les calculs inutiles.

In [5]:
from itertools import product

def kmerList(allkmers):
    validKmers = []
    for i in allkmers:
        kmer = "".join(i)
        validKmers.append(kmer)
    return validKmers


def removeLowComplexe(motifs, size):
    """
    Eleve les motifs peu complexe ayant 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie motifsClean: dictionnaire de motifs sans les motifs peu complexe.
    """
    motifsClean = []
    twoLetterCombs = ["".join(tup) for tup in list(product(["A", "T", "G", "C"], repeat=2))]
    for motif in [m.upper() for m in motifs]:
        if motif.count("A") > size or motif.count("T") > size or motif.count("G") > size or motif.count("C") > size:
            continue
        if True in [x*3 in motif for x in twoLetterCombs]:
            continue
        motifsClean.append(motif)
        
    return motifsClean



allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)

print (len(kmers))
kmersValid = removeLowComplexe(kmers, 5)
print (len(kmersValid))
print (kmersValid[0])

kmersValid = removeLowComplexe(kmersValid, 5)
print (len(kmersValid))

65536
63948
AAAAACCC
63948


4\. **Exact matching:** Définissez la fonction ``exact_match`` qui cherche dans le suffix tree tous les motifs possibles (k-mers), générés à la question precedent. La fonction renvoie un dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrence (valeurs). Ce dictionnaire doit être trié par nombre d'occurrences. 

Ensuite, trouvez et affichez les 10 motifs plus fréquents dans notre jeux de données artificiels.

In [6]:
def exact_match(kmersV, stree):
    """
    Cherche dans le suffix tree tous les motifs possibles
    entrée1 kmersV: Liste de Kmers à chercher
    entrée2 stree: suffix tree
    sortie1 motif_occur_sorted: dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrences (values).
    """
    motif_occur_sorted = {}

    for i in kmersV:
        motif_occur_sorted[i] = len(stree.find_all(i))

    return dict(sorted(motif_occur_sorted.items(), key=lambda x: x[1], reverse=True))

st = construct_tree(adn)
motif_occur_sorted = exact_match(kmersValid, st)
motif_occur_sorted


{'ATAGTCGA': 4,
 'AATAGTCG': 3,
 'AAATAGTC': 2,
 'AGCAGGCT': 2,
 'CGAGCAGG': 2,
 'GAGCAGGC': 2,
 'TAGTCGAA': 2,
 'TAGTCGAG': 2,
 'AAACTGAA': 1,
 'AAAGCTCA': 1,
 'AACAAGGA': 1,
 'AACAGCTA': 1,
 'AACCCGCG': 1,
 'AACTGAAT': 1,
 'AAGAATGG': 1,
 'AAGATTTC': 1,
 'AAGCTCAG': 1,
 'AAGGACGT': 1,
 'AAGGATGA': 1,
 'AAGTGCTC': 1,
 'AATACACT': 1,
 'AATATTAC': 1,
 'AATCAGAA': 1,
 'AATCTCAA': 1,
 'AATGGGCT': 1,
 'AATTTCGT': 1,
 'AATTTTTC': 1,
 'ACAACAAG': 1,
 'ACAAGGAC': 1,
 'ACAGCAAT': 1,
 'ACAGCTAC': 1,
 'ACAGTTCC': 1,
 'ACATATAG': 1,
 'ACCAGATT': 1,
 'ACCATCTT': 1,
 'ACCCGAAT': 1,
 'ACCCGCGC': 1,
 'ACCCTGGA': 1,
 'ACCTTGTC': 1,
 'ACGCCATG': 1,
 'ACGTGGCC': 1,
 'ACGTTACA': 1,
 'ACTACCAG': 1,
 'ACTCGGCC': 1,
 'ACTGAATT': 1,
 'ACTGTGAC': 1,
 'AGAACCCG': 1,
 'AGAATGGG': 1,
 'AGAGATTT': 1,
 'AGATGAAT': 1,
 'AGATTGTA': 1,
 'AGATTTAC': 1,
 'AGATTTCC': 1,
 'AGCAATCT': 1,
 'AGCTACAG': 1,
 'AGCTCAGG': 1,
 'AGCTCGAG': 1,
 'AGGACGTT': 1,
 'AGGACTAC': 1,
 'AGGATGAA': 1,
 'AGGCGTAG': 1,
 'AGGCTCGC': 1,
 'AGGCTT

5\. Introduisez deux variation (v=2) au motif initial. Pour cela avant de chaque implantation, créez d'abord un motif varié (avec v substitutions choisies aléatoirement) à partir du motif initial et puis implantez-le dans une séquence. Repetez pour chaque sequence dans le Jeux de donnée. Il suffit de mettre ``v`` égal ``2`` et réutiliser les fonctions définies à la question 1.

In [7]:
v=2
k=8 #taille de motif
v=0 #nb de positions variables dans le motif
t=5 #nb de sequences
n=100 #longuer des sequence
f=0.9

def implantMotifVar2(motif:str, v:int, t:int, n:int, f:float):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée motif: notre motif
    entrée v: variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f: frequence of the variation
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    ##adn false or true, parce que c'est pas grave pour notre TME si on travaille avec des sequences pregenerés ou on les genere ici
    adnNew = []
    for i in range(t):
        adnNew.append(generateRandomSequence(n - len(motif), True))
    sequences = adnNew

    print(motif.upper())
    motif_new = motif
    for i in range(t):
        base_seq = sequences[i]
        if random.random() < f:
            motif_new = modifierMotif(motif, v, True)
        sequences[i] = insertMotif(base_seq, motif_new, random.choice(range(len(base_seq))))
        motif_new = motif
    
    return sequences

adn = implantMotifVar2(fix_motif, v, t, n, f)
adn

TGAAAATG


['CCACTCCATACGCAATGACGTGTGATTGCTTGTTAGACCAGATTACAAGTGCACGCCAACTGTGAAAGGGGACTGAAAATGAACTCGGATAGATCACGGT',
 'TGATATTCCAGCTAACACGTATGGACGGGGAAACTGAAAATGGCGTGGAATATTCTAAGCCATGCGGTTCGCCCGTTCTCTATATAAGAGCCATGCACTT',
 'TTGTTGAATAGCGTCTATCATCGGGTACATTATGAGAGGAACAAGAGGAAGAGGACGGTTGGCGCTTATAACCCGGCAATGAAAATGCCCCGCATGCGGC',
 'CGCCGCTAGCACGACCCCGTCAATGGCACCGGAACCAAGCTTATAGCAGACCTGTACTTCGCAGGTCTTGAAAATGCACAGGTGCAATGGAACAATATAG',
 'GACGACAATTATTGCTCCACCTACCGTGAGAGGGAGAAAAAGGCTCACTATAACCACCACACGTGTCCCGCTGCTGCGATGAAAATGGGAGGGTCTAATT']

6\. Construisez le suffix tree à nouveau à partir des nouvelles séquences en utilisant le python package suffix-trees.

In [8]:
st = construct_tree(adn)

7\. **Inexact matching:** 

Définissez fonction ``inexact_match`` qui cherche tous les motifs possibles (k-mers) générés à la question 2 dans le nouveau suffix tree donné (construit à partir des nouvelle séquences qui incluent le motif varié), et renvoie un dictionnaire qui contient les motifs (keys) et les listes de toutes leurs variations (values) ainsi que le meilleur motif variable. Il faut que vous utilisiez la technique *seed* pour trouver le motif variable. 

Ensuite, affichez le meilleur motif variable avec toutes son variation dans notre nouveaux jeux de données artificiels.
***Transformer en deux fonction separer, une pour for motif et autre for seed***

In [9]:
from scipy.spatial import distance
from collections import OrderedDict

def inexact_match(kmersV, sequences, stree, v):
    """
    cherche de motif variables dans un suffix tree
    entrée1 kmersV: liste de motifs à chercher
    entrée2 sequences: matrice de dimension txn avec les sequences
    entrée3 stree: suffix Tree
    entrée4 v: nombre de variations dans le motifs
    sortie1 motif_occur_sorted: dictionnaire clés=sequence consensus des motifs; value= nombre d'occurrences.
    """
    sequencesNew = ""
    for sequence in sequences:
        sequencesNew += sequence

    motif_occur_sorted = {}
    motif_var = {}

    Ns = v + 1
    Ls = len(kmersV[0])//Ns
    for motif in kmersV:
        for i in range(0, Ns):
            seed = motif[i*Ls:i*Ls + Ls]
            candidateIndex = stree.find_all(seed)
            for index in candidateIndex:
                candidateText = sequencesNew[index - i*Ls:index+len(motif) - i*Ls]
                if len(candidateText) == len(motif) and round(distance.hamming(list(motif), list(candidateText)) * len(list(motif))) <= v:
                    if (candidateText in motif_occur_sorted.keys()): 
                        motif_occur_sorted[candidateText].add(index)
                    else:
                        motif_occur_sorted[candidateText] = set()
                        motif_occur_sorted[candidateText].add(index)
                    if (motif in motif_var.keys()):
                        motif_var[motif].add(candidateText)
                    else:
                        motif_var[motif] = set()
                        motif_var[motif].add(candidateText)
    
    return dict(OrderedDict(sorted(motif_occur_sorted.items(), key = lambda x : len(x[1])))), dict(OrderedDict(sorted(motif_var.items(), key = lambda x : len(x[1]))))

#Test

# seqTest = "banananabanbnaabanbna"
# stNew = construct_tree(seqTest)
# k=8
# motif_occur_sorted, motif_var = inexact_match(['banbna'], [seqTest], stNew, 1)
st = construct_tree(adn)
motif_occur_sorted, motif_var = inexact_match([fix_motif], adn, st, 2)
print(motif_var, motif_occur_sorted)


{'TGAAAATG': {'TGAAAGGG', 'TGAAAATG', 'GGAAACTG'}} {'GGAAACTG': {130}, 'TGAAAGGG': {64, 62}, 'TGAAAATG': {134, 136, 73, 138, 75, 77, 368, 370, 372, 279, 281, 283}}


8\. Créez le motif logo à partir des séquences du meilleur motif variable que vouz venez de trouver. Vous pouvez utilizer ce site: https://weblogo.berkeley.edu/logo.cgi. Affichez votre logo ci-dessous.

9\. Avez-vous réussi à trouver votre motif initial implanté en séquences? l'algorithme était-il rapide? Quelle est la complexité de chaque recherche de motif?

Oui, parmi les variations trouvé le plus frequant était notre motif initial, l'algorithme était rapide, compléxité de recherche dans un arbre est linéere

10\. Tester l'algorithme  suffix tree sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [10]:
def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie séquences: liste contenant toutes les séquences du fichier
    """

    sequence = ""
    sequences_list = []
    prev_header = ""
    header = ""

    for line in open(fastaFileName):
        string = line.strip()
        if string[0] != ">":
            if prev_header != header:
                prev_header = header
            sequence = sequence + string
        else:
            header = string
            if sequence != "":
                sequences_list.append(sequence.upper())
                sequence = ""

    sequences_list.append(sequence.upper())

    return sequences_list
genome = "Sequence_by_Peaks_3.fasta"
sequences = readFasta(genome)
#print(sequences)

In [11]:
k = 5
allkmers = product(nuc, repeat=k)
kmers = [''.join(kmer) for kmer in list(allkmers)]

KMERS_VALID = removeLowComplexe(kmers, 5)
KMERS_VALID[:10]

['AAAAA',
 'AAAAC',
 'AAAAG',
 'AAAAT',
 'AAACA',
 'AAACC',
 'AAACG',
 'AAACT',
 'AAAGA',
 'AAAGC']

In [12]:
st = construct_tree(sequences)
motif_occur_sorted, motif_var = inexact_match(KMERS_VALID, adn, st, 2)
motif_var, motif_occur_sorted

({'TTTTG': {'CTGTG',
   'CTTCG',
   'GGTTG',
   'TATAG',
   'TATTG',
   'TCTTG',
   'TGTTG',
   'TTATA',
   'TTATG',
   'TTATT',
   'TTCTC'},
  'CGTCC': {'ACTCC',
   'AGACC',
   'CATGC',
   'CCACC',
   'CGCCG',
   'CGTCT',
   'CGTGA',
   'CGTGT',
   'CTACC',
   'CTTCG',
   'GGTCT',
   'TGTAC'},
  'TATTT': {'AATTA',
   'CACTT',
   'TACTT',
   'TATAA',
   'TATAG',
   'TATAT',
   'TATGA',
   'TATTG',
   'TCTAT',
   'TCTCT',
   'TCTTG',
   'TGTTG'},
  'TTTTC': {'ATTGC',
   'CGTTC',
   'CTATC',
   'TATTG',
   'TCTTG',
   'TGTAC',
   'TGTTG',
   'TTATA',
   'TTATG',
   'TTATT',
   'TTCGC',
   'TTCTC'},
  'AAGTC': {'AAATG',
   'AAGAG',
   'AAGGG',
   'AATTA',
   'ACGTA',
   'AGGAC',
   'AGGTC',
   'ATGGC',
   'CAGAC',
   'CCGTC',
   'GGGTC',
   'GTGTC',
   'TAGAC'},
  'ATTTT': {'AATAT',
   'ACTGT',
   'ACTTT',
   'AGCTT',
   'ATATA',
   'ATTAT',
   'ATTGC',
   'CTTAT',
   'CTTGT',
   'GTCTT',
   'GTTCT',
   'TTATT',
   'TTGTT'},
  'TCATC': {'ACTTC',
   'AGATC',
   'CCACC',
   'CCATG',
   'GCA

In [13]:
for i in motif_var.items():
    print(i)

for i in motif_occur_sorted.items():
    print(i)

('TTTTG', {'CTGTG', 'TTATA', 'TCTTG', 'TTATT', 'CTTCG', 'TATTG', 'GGTTG', 'TGTTG', 'TATAG', 'TTATG', 'TTCTC'})
('CGTCC', {'CGTCT', 'CGTGA', 'CGCCG', 'CTACC', 'CTTCG', 'AGACC', 'CGTGT', 'ACTCC', 'TGTAC', 'GGTCT', 'CATGC', 'CCACC'})
('TATTT', {'AATTA', 'TCTTG', 'TATAA', 'TATTG', 'TATGA', 'TACTT', 'TATAT', 'TGTTG', 'TATAG', 'CACTT', 'TCTAT', 'TCTCT'})
('TTTTC', {'CTATC', 'CGTTC', 'TCTTG', 'TTATA', 'TTATT', 'ATTGC', 'TATTG', 'TTCGC', 'TGTAC', 'TGTTG', 'TTATG', 'TTCTC'})
('AAGTC', {'AAATG', 'ATGGC', 'CAGAC', 'ACGTA', 'AATTA', 'GGGTC', 'AAGAG', 'AGGAC', 'GTGTC', 'AGGTC', 'CCGTC', 'TAGAC', 'AAGGG'})
('ATTTT', {'GTTCT', 'ACTTT', 'ATTGC', 'CTTAT', 'TTATT', 'AATAT', 'CTTGT', 'AGCTT', 'TTGTT', 'ATTAT', 'GTCTT', 'ACTGT', 'ATATA'})
('TCATC', {'TATTC', 'TGAAC', 'ACTTC', 'GCATG', 'TTATA', 'TCTTG', 'TGCTC', 'TCACT', 'TAACC', 'AGATC', 'TAAGC', 'CCACC', 'CCATG'})
('TCGTC', {'TATTC', 'ACTTC', 'TCTTG', 'GGGTC', 'ACGTG', 'TCGCA', 'ACGAC', 'CCGGC', 'TCCAC', 'TTGTT', 'TAGAC', 'TCGGA', 'CCGTG'})
('TTATG', {'A